In [2]:
import os
import json
import requests
import pandas as pd
from pathlib import Path

In [93]:
pd.options.display.max_rows=400
pd.options.display.max_colwidth=None

In [28]:
poi_file = 'Data/opencharge_poi_data.json'

if Path(poi_file).exists():
    poi_df = pd.read_json(poi_file, encoding='utf8')

poi_df.head()

,IsRecentlyVerified,DateLastVerified,ID,UUID,DataProviderID,OperatorID,UsageTypeID,UsageCost,AddressInfo,Connections,...,StatusTypeID,DateLastStatusUpdate,DataQualityLevel,DateCreated,SubmissionStatusTypeID,DataProvidersReference,DatePlanned,OperatorsReference,MetadataValues,DateLastConfirmed
0,True,2024-10-07T17:20:00Z,304959,CA4DD728-806C-476F-9DEA-482CABAF83A9,1,3343.0,1.0,$0.35/kWh,"{'ID': 305348, 'Title': 'AutoNation Ford Corpus Christi', 'AddressLine1': '6250 S Padre Island D...","[{'ID': 577200, 'ConnectionTypeID': 32, 'StatusTypeID': 50, 'LevelID': 3, 'PowerKW': 120, 'Curre...",...,50.0,2024-10-07T17:20:00Z,1.0,2024-10-07T16:28:00Z,200,NaN,NaN,NaN,NaN,NaN
1,True,2024-10-07T17:20:00Z,304955,3C10CBCB-8C82-47AE-A0D3-8008632BC5AB,1,5.0,4.0,NaN,"{'ID': 305344, 'Title': '5140 Business Center Drive', 'AddressLine1': '5140 Business Center Driv...","[{'ID': 577191, 'ConnectionTypeID': 1, 'StatusTypeID': 50, 'LevelID': 2, 'Amps': 32, 'Voltage': ...",...,50.0,2024-10-07T17:20:00Z,1.0,2024-10-06T21:49:00Z,200,NaN,NaN,NaN,NaN,NaN
2,True,2024-10-07T17:23:00Z,304949,4FB87A99-81A9-4B60-84BB-B7C93EE25DFE,1,3534.0,4.0,NaN,"{'ID': 305338, 'Title': 'Rove - Santa Ana', 'AddressLine1': '1008 17th Street', 'Town': 'Santa A...","[{'ID': 577183, 'ConnectionTypeID': 27, 'StatusTypeID': 50, 'CurrentTypeID': 30, 'Quantity': 28}...",...,50.0,2024-10-07T17:23:00Z,1.0,2024-10-06T01:21:00Z,200,NaN,NaN,NaN,NaN,NaN
3,True,2024-10-07T17:24:00Z,304947,CF6F3A6A-4858-4A94-9022-7826A8C84D44,1,5.0,4.0,NaN,"{'ID': 305336, 'Title': 'Arther F Turner Community Library', 'AddressLine1': '1075 W Capital', '...","[{'ID': 577178, 'ConnectionTypeID': 1, 'StatusTypeID': 50, 'LevelID': 2, 'Amps': 32, 'Voltage': ...",...,50.0,2024-10-07T17:24:00Z,1.0,2024-10-05T20:34:00Z,200,NaN,NaN,NaN,NaN,NaN
4,True,2024-10-07T17:23:00Z,304934,228DE4C5-C712-47E0-8E2E-1314469A6CE1,1,3372.0,4.0,NaN,"{'ID': 305323, 'Title': 'Bowlero Manteca', 'AddressLine1': '1251 E Yosemite Ave', 'Town': 'Mante...","[{'ID': 577154, 'ConnectionTypeID': 32, 'StatusTypeID': 50, 'LevelID': 3, 'PowerKW': 180, 'Curre...",...,50.0,2024-10-07T17:23:00Z,1.0,2024-10-05T00:53:00Z,200,NaN,NaN,NaN,NaN,NaN


In [30]:
# Extract Address Elements from AddressInfo Dictionary
def extract_values(dictionary):
    ID = dictionary['ID']
    Title = dictionary.get('Title','')
    AddressLine1 = dictionary.get('AddressLine1', '')
    AddressLine2 = dictionary.get('AddressLine2', '')    
    Town = dictionary.get('Town', '')
    StateOrProvince = dictionary.get('StateOrProvince', '')
    Postcode = dictionary.get('Postcode', '')    
    return ID, Title, AddressLine1, AddressLine2, Town, StateOrProvince, Postcode
    
    
poi_df[['AddressInfoID', 'Title', 'AddressLine1', 'AddressLine2', 'Town', 'StateOrProvince', 'Postcode']] = poi_df['AddressInfo'].apply(lambda x: pd.Series(extract_values(x)))

poi_df.head()


,IsRecentlyVerified,DateLastVerified,ID,UUID,DataProviderID,OperatorID,UsageTypeID,UsageCost,AddressInfo,Connections,...,OperatorsReference,MetadataValues,DateLastConfirmed,AddressInfoID,Title,AddressLine1,AddressLine2,Town,StateOrProvince,Postcode
0,True,2024-10-07T17:20:00Z,304959,CA4DD728-806C-476F-9DEA-482CABAF83A9,1,3343.0,1.0,$0.35/kWh,"{'ID': 305348, 'Title': 'AutoNation Ford Corpus Christi', 'AddressLine1': '6250 S Padre Island D...","[{'ID': 577200, 'ConnectionTypeID': 32, 'StatusTypeID': 50, 'LevelID': 3, 'PowerKW': 120, 'Curre...",...,NaN,NaN,NaN,305348,AutoNation Ford Corpus Christi,6250 S Padre Island Drive,,Corpus Christi,TX,78412-4012
1,True,2024-10-07T17:20:00Z,304955,3C10CBCB-8C82-47AE-A0D3-8008632BC5AB,1,5.0,4.0,NaN,"{'ID': 305344, 'Title': '5140 Business Center Drive', 'AddressLine1': '5140 Business Center Driv...","[{'ID': 577191, 'ConnectionTypeID': 1, 'StatusTypeID': 50, 'LevelID': 2, 'Amps': 32, 'Voltage': ...",...,NaN,NaN,NaN,305344,5140 Business Center Drive,5140 Business Center Drive,,Fairfield,CA,94534
2,True,2024-10-07T17:23:00Z,304949,4FB87A99-81A9-4B60-84BB-B7C93EE25DFE,1,3534.0,4.0,NaN,"{'ID': 305338, 'Title': 'Rove - Santa Ana', 'AddressLine1': '1008 17th Street', 'Town': 'Santa A...","[{'ID': 577183, 'ConnectionTypeID': 27, 'StatusTypeID': 50, 'CurrentTypeID': 30, 'Quantity': 28}...",...,NaN,NaN,NaN,305338,Rove - Santa Ana,1008 17th Street,,Santa Ana,CA,92701
3,True,2024-10-07T17:24:00Z,304947,CF6F3A6A-4858-4A94-9022-7826A8C84D44,1,5.0,4.0,NaN,"{'ID': 305336, 'Title': 'Arther F Turner Community Library', 'AddressLine1': '1075 W Capital', '...","[{'ID': 577178, 'ConnectionTypeID': 1, 'StatusTypeID': 50, 'LevelID': 2, 'Amps': 32, 'Voltage': ...",...,NaN,NaN,NaN,305336,Arther F Turner Community Library,1075 W Capital,,West Sacramento,CA,95691
4,True,2024-10-07T17:23:00Z,304934,228DE4C5-C712-47E0-8E2E-1314469A6CE1,1,3372.0,4.0,NaN,"{'ID': 305323, 'Title': 'Bowlero Manteca', 'AddressLine1': '1251 E Yosemite Ave', 'Town': 'Mante...","[{'ID': 577154, 'ConnectionTypeID': 32, 'StatusTypeID': 50, 'LevelID': 3, 'PowerKW': 180, 'Curre...",...,NaN,NaN,NaN,305323,Bowlero Manteca,1251 E Yosemite Ave,,Manteca,CA,95336


In [99]:
l=poi_df[poi_df.ID == 304934]['Connections']
print(l)



4    [{'ID': 577154, 'ConnectionTypeID': 32, 'StatusTypeID': 50, 'LevelID': 3, 'PowerKW': 180, 'CurrentTypeID': 30, 'Quantity': 1}, {'ID': 577155, 'ConnectionTypeID': 2, 'StatusTypeID': 50, 'CurrentTypeID': 30, 'Quantity': 4}]
Name: Connections, dtype: object


In [103]:
for idx in poi_df[poi_df.ID == 304934].index:
    for i in poi_df['Connections'][idx]:
        print(i)
    

{'ID': 577154, 'ConnectionTypeID': 32, 'StatusTypeID': 50, 'LevelID': 3, 'PowerKW': 180, 'CurrentTypeID': 30, 'Quantity': 1}
{'ID': 577155, 'ConnectionTypeID': 2, 'StatusTypeID': 50, 'CurrentTypeID': 30, 'Quantity': 4}


In [36]:
poi_df.columns

Index(['IsRecentlyVerified', 'DateLastVerified', 'ID', 'UUID',
       'DataProviderID', 'OperatorID', 'UsageTypeID', 'UsageCost',
       'AddressInfo', 'Connections', 'NumberOfPoints', 'GeneralComments',
       'StatusTypeID', 'DateLastStatusUpdate', 'DataQualityLevel',
       'DateCreated', 'SubmissionStatusTypeID', 'DataProvidersReference',
       'DatePlanned', 'OperatorsReference', 'MetadataValues',
       'DateLastConfirmed', 'AddressInfoID', 'Title', 'AddressLine1',
       'AddressLine2', 'Town', 'StateOrProvince', 'Postcode'],
      dtype='object')

In [ ]:
poi_df.StateOrProvince.value_counts()

In [40]:
search='[A-Z][A_Z][A-Z]'

poi_df[poi_df.StateOrProvince.str.contains(search)].StateOrProvince.unique()

array(['CALIFORNIA'], dtype=object)

In [ ]:
pc = poi_df.Postcode.value_counts()
print(pc)

In [52]:
from pyzipcode import ZipCodeDatabase


In [63]:
zipdb=ZipCodeDatabase()
def return_state_from_zipcode(zipcode):
    if zipcode in zipdb:
        return zipdb[zipcode].state
    else:
        return ''

poi_df['NewState'] = poi_df.Postcode.apply(lambda x: return_state_from_zipcode(x))

poi_df[['StateOrProvince', 'Postcode', 'NewState']].sample(10)


,StateOrProvince,Postcode,NewState
34721,CA,90034,CA
23327,UT,84115,UT
39290,NC,28078,NC
38676,UT,84721,UT
3680,NC,28801,NC
43555,NC,27599,NC
49523,CO,80019,CO
585,MA,02155,MA
48159,CO,80621,CO
10122,CA,94065,CA


In [70]:
poi_df[poi_df.NewState != poi_df.StateOrProvince].sample(10)

,IsRecentlyVerified,DateLastVerified,ID,UUID,DataProviderID,OperatorID,UsageTypeID,UsageCost,AddressInfo,Connections,...,DateLastConfirmed,AddressInfoID,Title,AddressLine1,AddressLine2,Town,StateOrProvince,Postcode,valid_post_code,NewState
48587,False,2018-06-16T05:27:00Z,106130,9822C635-8ABC-4137-B29A-5948CFDF25F5,1,23.0,4.0,NaN,"{'ID': 106476, 'Title': 'Staybridge Suites Eagan Arpt South - Mall Area', 'AddressLine1': '4675 ...","[{'ID': 149462, 'ConnectionTypeID': 30, 'StatusTypeID': 50, 'LevelID': 2, 'PowerKW': 16, 'Curren...",...,NaN,106476,Staybridge Suites Eagan Arpt South - Mall Area,4675 Rahncliff RD,,Eagan,Minnesota,55122,True,MN
50224,False,2017-09-08T05:41:00Z,92692,4183B693-C50D-4C25-A01A-AB3A9013F94E,1,23.0,4.0,NaN,"{'ID': 93038, 'Title': 'Black Oak Casino Resort', 'AddressLine1': '19400 Tuolumne Road', 'Town':...","[{'ID': 131298, 'ConnectionTypeID': 30, 'StatusTypeID': 50, 'LevelID': 2, 'PowerKW': 17, 'Curren...",...,NaN,93038,Black Oak Casino Resort,19400 Tuolumne Road,,Tuolumne,California,95379,True,CA
16929,False,2024-02-20T09:43:00Z,240236,49A9BB41-B804-41A2-AC99-24574D74D5E1,2,5.0,1.0,NaN,"{'ID': 240620, 'Title': 'IRVINE CO OFC 5241 CALIF 01', 'AddressLine1': '5241 California Ave', ...","[{'ID': 396173, 'ConnectionTypeID': 1, 'LevelID': 2, 'Amps': 16, 'Voltage': 230, 'PowerKW': 3.7,...",...,NaN,240620,IRVINE CO OFC 5241 CALIF 01,5241 California Ave,,Irvine,CA,92617,False,
7421,False,2024-03-05T09:38:00Z,274973,3ABF7568-A9AB-487C-BB40-0CA0BBF19CFA,1,9.0,2.0,NaN,"{'ID': 275360, 'Title': 'North Avenue A', 'AddressLine1': 'North Avenue A', 'StateOrProvince': '...","[{'ID': 477276, 'ConnectionTypeID': 1, 'LevelID': 2, 'Amps': 16, 'Voltage': 230, 'PowerKW': 3.7,...",...,NaN,275360,North Avenue A,North Avenue A,,,Florida,33021,True,FL
57255,False,2017-11-04T14:04:00Z,13805,B66B4D7D-EAF4-416D-8ADC-4A554AE6C292,15,5.0,NaN,NaN,"{'ID': 13702, 'Title': '499 Linden Ave', 'AddressLine1': '499 Linden Ave', 'Town': 'Carpinteria'...","[{'ID': 11252, 'ConnectionTypeID': 1, 'Reference': 'J1772', 'LevelID': 2}]",...,NaN,13702,499 Linden Ave,499 Linden Ave,,Carpinteria,California,93013,True,CA
58049,False,2017-11-04T14:05:00Z,9074,87D9CB31-2B4A-4433-929F-C95B4A396C2C,15,5.0,NaN,NaN,"{'ID': 8971, 'Title': '14509 SE Newport Way', 'AddressLine1': '14509 SE Newport Way', 'Town': 'B...","[{'ID': 6736, 'ConnectionTypeID': 1, 'Reference': 'J1772', 'LevelID': 2}, {'ID': 9047, 'Connecti...",...,NaN,8971,14509 SE Newport Way,14509 SE Newport Way,,Bellevue,Washington,98006,True,WA
51205,False,2017-06-09T14:52:00Z,84909,E585DA15-B6C0-47B9-A4D2-4970835E51CD,1,23.0,4.0,NaN,"{'ID': 85255, 'Title': 'Courtyard Reno Downtown', 'AddressLine1': '1 Ballpark LN', 'Town': 'Reno...","[{'ID': 120653, 'ConnectionTypeID': 30, 'StatusTypeID': 50, 'LevelID': 2, 'PowerKW': 8, 'Current...",...,NaN,85255,Courtyard Reno Downtown,1 Ballpark LN,,Reno,Nevada,89501,True,NV
56535,False,2014-05-24T10:01:00Z,23335,31CFF04A-B56F-4274-B406-D6DF78C3A484,1,1.0,6.0,Free for customers,"{'ID': 23680, 'Title': 'Woodstock Station Inn and Brewery', 'AddressLine1': '135 Main Street', '...","[{'ID': 25213, 'ConnectionTypeID': 1, 'StatusTypeID': 50, 'LevelID': 2, 'Amps': 30, 'Voltage': 2...",...,NaN,23680,Woodstock Station Inn and Brewery,135 Main Street,,Woodstock,NH,,False,
49529,False,2018-01-21T17:24:00Z,97957,B6FF449A-4CB8-45AA-AF9D-2A4FAD389330,1,23.0,4.0,NaN,"{'ID': 98303, 'Title': '1345 Parking Garage', 'AddressLine1': '1345 6th Ave', 'Town': 'New York'...","[{'ID': 138865, 'ConnectionTypeID': 30, 'StatusTypeID': 50, 'LevelID': 2, 'PowerKW': 10, 'Curren...",...,NaN,98303,1345 Parking Garage,1345 6th Ave,,New York,New York,10105,True,NY
50317,False,2017-09-01T05:35:00Z,92272,4B4C9C09-DC72-4714-8FDE-F2142776D596,1,23.0,4.0,NaN,"{'ID': 92618, 'Title': 'Great Pines', 'AddressLine1': '4920 State Route 28', 'Town': 'Old Forge'...","[{'ID': 130815, 'ConnectionTypeID': 30, 'StatusTypeID': 50, 'LevelID': 2, 'PowerKW': 19, 'Curren...",...,NaN,92618,Great Pines,4920 State Route 28,,

In [58]:
poi_df[['StateOrProvince', 'Postcode']].sample(10)

,StateOrProvince,Postcode
27953,TN,38585
44177,CA,91304
54714,NJ,07059
52545,MA,01003
36980,NE,68127
18426,WA,98103
52492,California,94061
9285,WA,98225
16321,CA,91709
11295,MN,55121
